In [ ]:
import numpy as np
import pandas as pd
import pickle
import torch

from util import is_json, parse_har, write_har, postprocess_har
from trace_gen import process_event, gen_trace
from feature_gen import gen_feature
from gen_types import Event, System
from plotter import plot_frequencies

%load_ext autoreload
%autoreload 2

In [ ]:
torch.__version__

In [ ]:
# ---------------------
num_worker = 1
num_user = 1
mission_hour = 1
# ---------------------

mission_minutes = mission_hour*60
mission_seconds = mission_hour*60*60

# ---------------------

aggregate_har, sys = gen_trace(num_user, mission_minutes, num_worker)

In [ ]:
test_pdf = postprocess_har(aggregate_har)

f = open("user_{}_hour_{}_cs.uchicago.edu.pandas".format(num_user, mission_hour), "wb")
pickle.dump(test_pdf, f)
f.close()

In [ ]:
url_pdf = test_pdf.groupby('url').size().sort_values(ascending=False)
url_pdf.head(50)

In [ ]:
# Load the HAR dict pickle file
gatech_har = pickle.load(open("user_2000_hour_12_cc.gatech.edu.har", "rb"))

In [ ]:
gatech_processed = postprocess_har(gatech_har)

In [ ]:
pd.set_option('display.max_rows', None)

url_pdf = gatech_processed.groupby('url').size().sort_values(ascending=False)
print(url_pdf)

In [ ]:
plot_urls = [
    "https://use.typekit.net/ymn0bgw.css",
    "https://kit.fontawesome.com/37926dae7e.js",
    "https://www.cc.gatech.edu/themes/contrib/gt_theme/logo.svg",
    "https://www.cc.gatech.edu/sites/default/files/js/js_cF0PQeAVLuB7uJgTjVNgmUrdrI_zUkpq25mHJnMbl-c.js"
    # "https://cs.uchicago.edu/wp-includes/css/dashicons.min.css?ver=5.9",
    # "https://cs.uchicago.edu/wp-content/uploads/2022/01/2653-1600x900-1-527x332.jpeg",
    # "https://cs.uchicago.edu/wp-content/themes/csuc/build/fonts/GothamNarrSSm-BookItalic_Web.woff2"
]

frequency_pdfs = plot_frequencies(
    gatech_processed,
    plot_urls,
    12*60*60,
    240,
    (0,7500),
    (0,3)
)

In [ ]:
plot_urls = [
    "https://use.typekit.net/ymn0bgw.css",
    "https://kit.fontawesome.com/37926dae7e.js",
    "https://www.cc.gatech.edu/themes/contrib/gt_theme/logo.svg",
    "https://www.cc.gatech.edu/sites/default/files/js/js_cF0PQeAVLuB7uJgTjVNgmUrdrI_zUkpq25mHJnMbl-c.js"
    # "https://cs.uchicago.edu/wp-includes/css/dashicons.min.css?ver=5.9",
    # "https://cs.uchicago.edu/wp-content/uploads/2022/01/2653-1600x900-1-527x332.jpeg",
    # "https://cs.uchicago.edu/wp-content/themes/csuc/build/fonts/GothamNarrSSm-BookItalic_Web.woff2"
]

frequency_pdfs = plot_frequencies(
    gatech_processed,
    plot_urls,
    12*60*60,
    240,
    (0,7500),
    (0,3)
)

In [ ]:
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def gen_train(freq_pdf, seq_length):
    series = freq_pdf['count_rolling'].to_numpy()
    xs, ys = [], []
    for i in range(len(series) - seq_length - 1):
        xs.append(series[i:(i+seq_length)])
        ys.append(series[i+seq_length])
        
    return torch.from_numpy(np.array(xs)).float().to(device).unsqueeze(-1), torch.from_numpy(np.array(ys)).float().to(device)

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, output_size)
        
        for name, param in self.lstm.named_parameters():
            if 'bias' in name:
                nn.init.constant(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_normal(param)
                
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = 1
hidden_size = 4
num_layers = 2
output_size = 1
model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)

learning_rate = 1e-3
num_epochs = 20

criteria = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

window_size=200
X, Y = gen_train(frequency_pdfs[0], window_size)

In [ ]:
test_np = frequency_pdfs[0]['count_rolling'].to_numpy()
test_np[4200:4210]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

dataset_train = TensorDataset(X, Y)
dataloader = DataLoader(dataset_train, batch_size=512, shuffle=True)

In [ ]:
losses = []
for epoch in range(num_epochs):
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):
        # print(y_batch)
        outputs = model(x_batch)
        # print(outputs)
        
        y_batch = y_batch.reshape(outputs.shape)
        
        optimizer.zero_grad()
        loss = criteria(outputs, y_batch) * 200
        loss.backward()
        optimizer.step()
        
    print("Epoch: {}/{}, Loss: {:.8f}".format(epoch+1, num_epochs, loss.item()))
    losses.append(loss.item())

import matplotlib.pyplot as plt

plt.plot(range(0, num_epochs), losses)

In [ ]:
def infer_plot_pred(test_pdf):
    test_X, test_Y = gen_train(test_pdf, window_size)

    dataset_test = TensorDataset(X)
    dataloader_test = DataLoader(dataset_test, batch_size=64)

    y_test_preds = []
    for id_batch, (x_batch) in enumerate(dataloader_test):
        y_pred = model(x_batch[0])
        y_pred = y_pred.flatten().cpu().tolist()
        y_test_preds += y_pred
    
    plt.figure(figsize=(30, 4), dpi=200)
    plt.plot(test_pdf['count_rolling'], label="True Freq")
    plt.plot([0]*window_size + y_test_preds, label="Prediction")
    plt.xlim((0,7500))
    plt.ylim((0, 3))
    plt.legend(loc='upper right')

infer_plot_pred(frequency_pdfs[0])
infer_plot_pred(frequency_pdfs[1])

In [ ]:
plot_urls = [
    "https://cs.uchicago.edu/wp-content/themes/csuc/build/fonts/GothamNarrSSm-Black_Web.woff2",
    "https://cs.uchicago.edu/",
    # "https://cs.uchicago.edu/wp-includes/css/dashicons.min.css?ver=5.9",
    # "https://cs.uchicago.edu/wp-content/uploads/2022/01/2653-1600x900-1-527x332.jpeg",
    # "https://cs.uchicago.edu/wp-content/themes/csuc/build/fonts/GothamNarrSSm-BookItalic_Web.woff2"
]

plot_frequencies(
    test_pdf,
    plot_urls,
    mission_seconds,
    240
)

In [ ]:
load_path = "../data/cs.uchicago.edu/"
fpd = gen_feature(load_path)
fpd.head(5)